In [13]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import recall_score, classification_report

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
## defining input and target variables
x = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y = heart['TenYearCHD']

## splitting the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [5]:
x_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3344.000000,3374.000000,3390.000000
mean,49.631563,0.491445,236.584629,25.817152,75.814749
std,8.519895,0.500001,44.113585,4.086519,12.106316
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.080000,68.000000
50%,49.000000,0.000000,233.000000,25.380000,75.000000
75%,56.000000,1.000000,263.000000,28.040000,83.000000
max,70.000000,1.000000,600.000000,56.800000,143.000000


In [6]:
x_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,844.000000,845.000000,847.000000
mean,49.398585,0.504717,237.264218,25.741538,76.135773
std,8.780632,0.500273,46.454177,4.056267,11.705578
min,33.000000,0.000000,119.000000,15.960000,48.000000
25%,42.000000,0.000000,205.000000,23.050000,68.000000
50%,48.500000,1.000000,235.000000,25.450000,75.000000
75%,57.000000,1.000000,264.250000,28.020000,84.000000
max,69.000000,1.000000,696.000000,45.800000,130.000000


In [7]:
## defining the imputer
imputer = KNNImputer(n_neighbors = 5, weights = 'distance').fit(x_train)

## impute the missing values
x_train_imp = pd.DataFrame(imputer.transform(x_train), columns = x_train.columns)
x_test_imp = pd.DataFrame(imputer.transform(x_test), columns = x_test.columns)

In [8]:
x_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.631563,0.491445,236.559611,25.815276,75.814749
std,8.519895,0.500001,43.924151,4.080233,12.106316
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.080000,68.000000
50%,49.000000,0.000000,234.000000,25.380000,75.000000
75%,56.000000,1.000000,262.000000,28.040000,83.000000
max,70.000000,1.000000,600.000000,56.800000,143.000000


In [10]:
x_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.398585,0.504717,237.156432,25.744808,76.135863
std,8.780632,0.500273,46.400660,4.050838,11.698666
min,33.000000,0.000000,119.000000,15.960000,48.000000
25%,42.000000,0.000000,205.000000,23.057500,68.000000
50%,48.500000,1.000000,235.000000,25.450000,75.000000
75%,57.000000,1.000000,264.000000,28.025000,84.000000
max,69.000000,1.000000,696.000000,45.800000,130.000000


In [16]:
## building models
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(x_train_imp, y_train)
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(x_train_imp, y_train)

## predicting on the testing dataset
ada_preds = ada_md.predict_proba(x_test_imp)[:, 1]
rf_preds = rf_md.predict_proba(x_test_imp)[:, 1]

## changing liklihoods to labels
ada_labels = np.where(ada_preds < 0.1, 0, 1)
rf_labels = np.where(rf_preds < 0.1, 0, 1)

## printing classification report LOGIT
print('AdaBoost Classifier:')
print(classification_report(y_test, ada_labels))

## printing classification report RF
print('Random Forest:')
print(classification_report(y_test, rf_labels))

AdaBoost Classifier:
              precision    recall  f1-score   support

           0       1.00      0.00      0.01       719
           1       0.15      1.00      0.26       129

    accuracy                           0.16       848
   macro avg       0.58      0.50      0.14       848
weighted avg       0.87      0.16      0.05       848

Random Forest:
              precision    recall  f1-score   support

           0       0.93      0.39      0.55       719
           1       0.20      0.84      0.32       129

    accuracy                           0.46       848
   macro avg       0.56      0.61      0.43       848
weighted avg       0.82      0.46      0.51       848



Based on the above classification reports, I would chose Random Forest to predict TenYearCHD.